In [11]:
import cv2
import cv2.legacy
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

In [12]:
#Input demo video name.
video_fileName = 'demo.mp4'
### Initialize and select tracking algorithm
(major_ver, minor_ver, subminor_ver) = cv2.__version__.split('.')

# 定义全局变量
selected_points = []  # 存储用户框选的角点
tracking_points = []  # 存储正在追踪的角点


#选择跟踪器-取决于类型的应用程序和类型的运动跟踪(高频，大振幅，等等…)
tracker_types = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[1]

if tracker_type == 'BOOSTING':
    tracker = cv2.TrackerBoosting_create()
if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()
if tracker_type == 'TLD':
    tracker = cv2.TrackerTLD_create()
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create()
if tracker_type == 'GOTURN':
    tracker = cv2.TrackerGOTURN_create()
if tracker_type == 'MOSSE':
    tracker = cv2.TrackerMOSSE_create()
if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

In [13]:
# 定义全局变量
selected_points = []  # 存储用户框选的角点
tracking_points = []  # 存储正在追踪的角点

In [14]:
# 框选感兴趣区域
def select_roi(event,x,y,flags,param):
    global roi_pts,selecting_roi,frame,frame_copy

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_pts = [(x,y)]
        selecting_roi = True
    elif event == cv2.EVENT_MOUSEMOVE and selecting_roi:
        frame_copy = frame.copy()
        cv2.rectangle(frame_copy,roi_pts[0],(x,y),(0,255),2)
        cv2.imshow("Select ROI",frame_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        roi_pts.append((x,y))
        selecting_roi = False
        cv2.rectangle(frame_copy,roi_pts[0],roi_pts[1],(0,255,0),2)
        cv2.imshow("Select ROI", frame_copy)


In [15]:
# 角点检测并绘制在视频上
def detect_and_draw_corners(frame,roi_pts):
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    selected_roi = frame[roi_pts[0][1]:roi_pts[1][1],roi_pts[0][0]:roi_pts[1][0]]
    gray_roi = cv2.cvtColor(selected_roi,cv2.COLOR_BGR2GRAY)

    # 检测角点
    corners = cv2.goodFeaturesToTrack(gray_roi, 100, 0.01, 10)
    corners = np.int0(corners)

    for corner in corners:
        x, y = corner.ravel()
        x += roi_pts[0][0]
        y += roi_pts[0][1]
        cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)

    # 添加检测的角点到列表中
    selected_points.append((x,y))

    return frame